In [120]:
from sympy.logic.boolalg import And, Or, Not, Implies, Equivalent
from sympy import symbols, bool_map
from sympy.logic.inference import satisfiable
from itertools import combinations
from belief_agent import BeliefBase
from math import factorial

**KB: Robert does well in the exam if and only if he is prepared or lucky.**
Robert does not do well the exam.

**Question: KB |= φ?**  **(KB entails phi)**

1. (-r $\vee$ p $\vee$ s) $\wedge$ (p $\wedge$ r) $\wedge$ (s $\wedge$ r) $\wedge$ $\neg$ r, formalisation of KB as CNF.

2.(-r $\vee$ p $\vee$ s) $\wedge$ (p $\wedge$ r) $\wedge$ (s $\wedge$ r) $\wedge$ r $\wedge$ p, CNF for contradiction.

3. clauses:= (-r $\vee$ p $\vee$ s), (p $\wedge$ r), (s $\wedge$ r), r, p.

4. clauses:=clauses U {r} from resolving (¬p $\vee$ r) with p.

5. clauses:=clauses U {1} from resolving r with $\neg$ r.

6. Result: empty clause, so KB |= φ.

In [132]:
belief_base = BeliefBase('q p')
symb = belief_base.symbols
q, p = symb

belief_base.expand(p)
belief_base.expand(q)
belief_base.expand(Implies(p, q))
print("Belief Base:", belief_base)

Belief Base: {q, p, q | ~p}


In [133]:
belief_base.priorities

{(True, True): 0, (True, False): 1, (False, True): 2, (False, False): 3}

In [148]:
satisf = satisfiable(belief_base.to_CNF())
key = tuple([satisf[belief_base.symbols[i]] for i in range(len(belief_base.symbols))])

In [149]:
belief_base.priorities(key)

TypeError: 'dict' object is not callable